In [1]:
# dependencies

import gmaps
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

In [2]:
ed_data = pd.read_csv("../resources/NEDI2018statedata.csv")
ed_data.head()

,Name,Abbreviation,Total # Eds,Total ED visit volume,Median visit volume,IQR visit volume,Unnamed: 6,<10k,10-19k,20-29k,30-39k,40-49k,>50k
0,Alaska,AK,20,"261,985","5,979","2,270","16,180",13,3,0,3,0,1
1,Alabama,AL,98,"2,825,756","22,197","11,000","42,087",18,28,11,15,8,18
2,Arkansas,AR,79,"1,730,571","12,350","7,000","32,799",30,19,9,5,4,12
3,Arizona,AZ,88,"3,119,364","29,200","12,775","50,000",14,20,13,10,8,23
4,California,CA,341,"15,787,403","41,975","24,000","67,630",37,36,37,54,44,133


In [3]:
mortality_data = pd.read_csv("../resources/us-counties.csv")
mortality_data.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [4]:
unique_value = mortality_data["state"].nunique()
unique_value

55

In [5]:
ed_data["Name"].nunique()

51

In [6]:
# groupby mortality_data on state name with sum on deaths

grp_mortality = mortality_data.drop(columns=["fips", "date"])

grp_mortality = grp_mortality.groupby(["state"]).sum().reset_index(["state"])
grp_mortality = grp_mortality.rename(columns={"state": "Name"})

#add death rate column death/cases
grp_mortality["Death Rate"] = round((grp_mortality["deaths"].astype(int)/grp_mortality["cases"].astype(int))*100,2)
grp_mortality.head()


,Name,cases,deaths,Death Rate
0,Alabama,46078611,728745.0,1.58
1,Alaska,4544475,20735.0,0.46
2,Arizona,69132576,1418028.0,2.05
3,Arkansas,27248041,428643.0,1.57
4,California,271972863,4177868.0,1.54


In [7]:
# remove virgin islands, guam, n mariana islands
# retain Puerto Rico
# merge dataframes on state name to retain the states in the ed_data
# this eliminates Puerto Rico from the dataset

merged_ed_mort = pd.merge(ed_data, grp_mortality, how = "left", on = "Name").sort_values("Name")
merged_ed_mort.head()

,Name,Abbreviation,Total # Eds,Total ED visit volume,Median visit volume,IQR visit volume,Unnamed: 6,<10k,10-19k,20-29k,30-39k,40-49k,>50k,cases,deaths,Death Rate
1,Alabama,AL,98,"2,825,756","22,197","11,000","42,087",18,28,11,15,8,18,46078611,728745.0,1.58
0,Alaska,AK,20,"261,985","5,979","2,270","16,180",13,3,0,3,0,1,4544475,20735.0,0.46
3,Arizona,AZ,88,"3,119,364","29,200","12,775","50,000",14,20,13,10,8,23,69132576,1418028.0,2.05
2,Arkansas,AR,79,"1,730,571","12,350","7,000","32,799",30,19,9,5,4,12,27248041,428643.0,1.57
4,California,CA,341,"15,787,403","41,975","24,000","67,630",37,36,37,54,44,133,271972863,4177868.0,1.54


In [8]:
# Read in census file

census_clean = pd.read_csv("../resources/census_clean.csv")

In [9]:
# Merge census data with the ED data

merged_all = pd.merge(merged_ed_mort, census_clean, how = "left", on = "Name").sort_values("Name")

In [10]:
merged_all = merged_all.drop(columns=['State', 'Median visit volume', 'IQR visit volume', 'Unnamed: 6', '<10k',
       '10-19k', '20-29k', '30-39k', '40-49k', '>50k'])
merged_all.head()

,Name,Abbreviation,Total # Eds,Total ED visit volume,cases,deaths,Death Rate,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,Alabama,AL,98,"2,825,756",46078611,728745.0,1.58,4876250.0,39.0,50536.0,27928.0,795989.0,16.323794
1,Alaska,AK,20,"261,985",4544475,20735.0,0.46,737068.0,34.3,77640.0,36787.0,76933.0,10.437707
2,Arizona,AZ,88,"3,119,364",69132576,1418028.0,2.05,7050299.0,37.7,58945.0,30694.0,1043764.0,14.804535
3,Arkansas,AR,79,"1,730,571",27248041,428643.0,1.57,2999370.0,38.1,47597.0,26577.0,496260.0,16.545475
4,California,CA,341,"15,787,403",271972863,4177868.0,1.54,39283497.0,36.5,75235.0,36955.0,5149742.0,13.109174


In [11]:
merged_all.columns

Index(['Name', 'Abbreviation', 'Total # Eds', 'Total ED visit volume', 'cases',
       'deaths', 'Death Rate', 'Population', 'Median Age', 'Household Income',
       'Per Capita Income', 'Poverty Count', 'Poverty Rate'],
      dtype='object')

In [12]:
merged_all = merged_all.rename(columns={"Name": "State",
                           "Total # Eds": "Total Emergency Depts",
                           "Total ED visit volume": "Total ED Visit Volume",
                          "cases": "Cases",
                          "deaths": "Deaths",
                          "Death Rate": "Death Rate (%)",
                           "Poverty Rate": "Poverty Rate (%)"})
merged_all.head()

,State,Abbreviation,Total Emergency Depts,Total ED Visit Volume,Cases,Deaths,Death Rate (%),Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate (%)
0,Alabama,AL,98,"2,825,756",46078611,728745.0,1.58,4876250.0,39.0,50536.0,27928.0,795989.0,16.323794
1,Alaska,AK,20,"261,985",4544475,20735.0,0.46,737068.0,34.3,77640.0,36787.0,76933.0,10.437707
2,Arizona,AZ,88,"3,119,364",69132576,1418028.0,2.05,7050299.0,37.7,58945.0,30694.0,1043764.0,14.804535
3,Arkansas,AR,79,"1,730,571",27248041,428643.0,1.57,2999370.0,38.1,47597.0,26577.0,496260.0,16.545475
4,California,CA,341,"15,787,403",271972863,4177868.0,1.54,39283497.0,36.5,75235.0,36955.0,5149742.0,13.109174


In [13]:
# Format the data for a final data frame
merged_all["Death Rate (%)"] = merged_all["Death Rate (%)"].map("{:.2f}".format)
merged_all["Poverty Rate (%)"] = merged_all["Poverty Rate (%)"].map("{:.2f}".format)
merged_all["Deaths"] = round(merged_all["Deaths"].astype(int), 0)
merged_all["Household Income"] = round(merged_all["Household Income"].astype(int), 0)
merged_all["Per Capita Income"] = round(merged_all["Per Capita Income"].astype(int), 0)
merged_all["Poverty Count"] = round(merged_all["Poverty Count"].astype(int), 0)
merged_all["Population"] = round(merged_all["Population"].astype(int), 0)

merged_all.head()

,State,Abbreviation,Total Emergency Depts,Total ED Visit Volume,Cases,Deaths,Death Rate (%),Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate (%)
0,Alabama,AL,98,"2,825,756",46078611,728745,1.58,4876250,39.0,50536,27928,795989,16.32
1,Alaska,AK,20,"261,985",4544475,20735,0.46,737068,34.3,77640,36787,76933,10.44
2,Arizona,AZ,88,"3,119,364",69132576,1418028,2.05,7050299,37.7,58945,30694,1043764,14.80
3,Arkansas,AR,79,"1,730,571",27248041,428643,1.57,2999370,38.1,47597,26577,496260,16.55
4,California,CA,341,"15,787,403",271972863,4177868,1.54,39283497,36.5,75235,36955,5149742,13.11


In [14]:
#add per capita emergency departments column to spreadsheet
merged_all["Per Capita EDs (100K)"] = round((merged_all["Total Emergency Depts"].astype(int)/(merged_all["Population"]).astype(int))*100000,2)
merged_all["Per Capita Cases"] = round((merged_all["Cases"].astype(int)/(merged_all["Population"]).astype(int))*100,2)
merged_all["Per Capita Deaths"] = round((merged_all["Deaths"].astype(int)/(merged_all["Population"]).astype(int))*100,2)
merged_all.head()

,State,Abbreviation,Total Emergency Depts,Total ED Visit Volume,Cases,Deaths,Death Rate (%),Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate (%),Per Capita EDs (100K),Per Capita Cases,Per Capita Deaths
0,Alabama,AL,98,"2,825,756",46078611,728745,1.58,4876250,39.0,50536,27928,795989,16.32,2.01,944.96,14.94
1,Alaska,AK,20,"261,985",4544475,20735,0.46,737068,34.3,77640,36787,76933,10.44,2.71,616.56,2.81
2,Arizona,AZ,88,"3,119,364",69132576,1418028,2.05,7050299,37.7,58945,30694,1043764,14.80,1.25,980.56,20.11
3,Arkansas,AR,79,"1,730,571",27248041,428643,1.57,2999370,38.1,47597,26577,496260,16.55,2.63,908.46,14.29
4,California,CA,341,"15,787,403",271972863,4177868,1.54,39283497,36.5,75235,36955,5149742,13.11,0.87,692.33,10.64


In [15]:
# Add per capita death rate
merged_all["Per Capita Death Rate"] = round((merged_all["Per Capita Deaths"].astype(int)/(merged_all["Per Capita Cases"]).astype(int))*100,2)
merged_all.head()

,State,Abbreviation,Total Emergency Depts,Total ED Visit Volume,Cases,Deaths,Death Rate (%),Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate (%),Per Capita EDs (100K),Per Capita Cases,Per Capita Deaths,Per Capita Death Rate
0,Alabama,AL,98,"2,825,756",46078611,728745,1.58,4876250,39.0,50536,27928,795989,16.32,2.01,944.96,14.94,1.48
1,Alaska,AK,20,"261,985",4544475,20735,0.46,737068,34.3,77640,36787,76933,10.44,2.71,616.56,2.81,0.32
2,Arizona,AZ,88,"3,119,364",69132576,1418028,2.05,7050299,37.7,58945,30694,1043764,14.80,1.25,980.56,20.11,2.04
3,Arkansas,AR,79,"1,730,571",27248041,428643,1.57,2999370,38.1,47597,26577,496260,16.55,2.63,908.46,14.29,1.54
4,California,CA,341,"15,787,403",271972863,4177868,1.54,39283497,36.5,75235,36955,5149742,13.11,0.87,692.33,10.64,1.45


In [17]:
# Export the data frame as a CSV
merged_all.to_csv(r'..\resources\data_final.csv', index = False, encoding='utf-8')